In [4]:
from bs4 import BeautifulSoup
import  requests

url = 'http://www.fedsdatacenter.com/federal-pay-rates/index.php'

#soup = BeautifulSoup(url)

r = requests.get(url)

soup = BeautifulSoup(r.text, "lxml")

#t = soup.find_all("table", class_="display Tables")

print(soup.find("table",{"class": "display"}))

<table class="display table table-striped table-bordered small" id="table-example" width="100%">
<thead>
<tr>
<th>Name</th>
<th>Grade</th>
<th>Pay Plan</th>
<th>Salary</th>
<th>Bonus</th>
<th>Agency</th>
<th>Location</th>
<th>Occupation</th>
<th>FY</th>
</tr>
</thead>
<tbody>
</tbody>
</table>


In [ ]:
from pandas.io.html import read_html
from selenium import webdriver


driver = webdriver.Firefox()
driver.get('http://www.fedsdatacenter.com/federal-pay-rates/index.php')

table = driver.find_element_by_xpath('//div[@class="display Tables"]/table//table/..')
table_html = table.get_attribute('outerHTML')

df = read_html(table_html)[0]
print df

driver.close()

In [21]:
"""
get max record count
"""
from jinja2 import Template
import requests
import json

num_display = 100

base_url = Template(
                    'http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&s'
                    'Echo=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart={{next_record_start}}&i'
                    'DisplayLength={{num_display}}&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1'
                    '=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&'
                    'mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&'
                    'bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true'
                    '&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082'
                    )

# base_url = Template("""
#                     http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&s
#                     Echo=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart={{next_record_start}}&i
#                     DisplayLength={{num_display}}&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1
#                     =true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&
#                     mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&
#                     bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true
#                     &iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082
#                     """.replace('\n', ''))

never_hit_record_count = 1000000000

u = base_url.render(next_record_start=never_hit_record_count, num_display=num_display)

print u

j = requests.get(u)

print j.json()

get_count = json.loads(j.text)
get_max_record_count = get_count['iTotalDisplayRecords']
print get_max_record_count
print int(get_max_record_count) / 100


http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&sEcho=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart=1000000000&iDisplayLength=100&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082
{u'aaData': [], u'iTotalRecords': u'8620477', u'sEcho': 1, u'iTotalDisplayRecords': u'1329915'}
1329915
13299


In [1]:
"""
Test data manipulation
"""
from jinja2 import Template
import requests
import json

num_display = 10

base_url = Template(
                    'http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&s'
                    'Echo=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart={{next_record_start}}&i'
                    'DisplayLength={{num_display}}&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1'
                    '=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&'
                    'mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&'
                    'bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true'
                    '&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082'
                    )

u = base_url.render(next_record_start=0, num_display=num_display)

print u

j = requests.get(u)

get_count = json.loads(j.text)
data = get_count['aaData']

def remove_erroneous_chars(data_to_clean):
    chars_to_remove = {'comma': ',', 'dollar': '$'}
    
    print data_to_clean
    separate_change = data_to_clean.split('.')
    final_sal = separate_change[0].replace(',', '')
    final_sal = final_sal.replace('$', '')
    cleaned_data = int(final_sal.replace('.', ''))
    print cleaned_data
        
    return cleaned_data

def clean_data(data):
    print "data before clean"
    print data
    for i in data:
        print i[3]
        i[3] = remove_erroneous_chars(i[3])
        print i[3]
        
        print i[4]
        i[4] = remove_erroneous_chars(i[4])
        print i[4]
        
        i[8] = int(i[8])
    
    print "data after clean"
    print data
    return data
    

clean_data(data)

http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&sEcho=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart=0&iDisplayLength=10&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082
data before clean
[[u'A VANT,MAY ARDAH LINDA', u'14', u'GS', u'$118,057.00', u'$1,000.00', u'CITIZENSHIP AND IMMIGRATION SERVICES', u'WASHINGTON', u'MANAGEMENT AND PROGRAM ANALYSIS', u'2015'], [u"A'ANO,TRACIE M", u'12', u'GS', u'$70,192.00', u'$0.00', u'INDIAN HEALTH SERVICE', u'SELLS', u'MEDICAL TECHNOLOGIST', u'2015'], [u"A'MAR,Z TERESA", u'03', u'ZP', u'$76,633.00', u'$0.00', u'NAT OCEANIC AND ATMOSPHERIC ADMIN', u'SEATTLE', u'OPERATIONS RESEARCH', u'2015'], [u'A,MINH D', u'14', u'GS', u'$

[[u'A VANT,MAY ARDAH LINDA',
  u'14',
  u'GS',
  118057,
  1000,
  u'CITIZENSHIP AND IMMIGRATION SERVICES',
  u'WASHINGTON',
  u'MANAGEMENT AND PROGRAM ANALYSIS',
  2015],
 [u"A'ANO,TRACIE M",
  u'12',
  u'GS',
  70192,
  0,
  u'INDIAN HEALTH SERVICE',
  u'SELLS',
  u'MEDICAL TECHNOLOGIST',
  2015],
 [u"A'MAR,Z TERESA",
  u'03',
  u'ZP',
  76633,
  0,
  u'NAT OCEANIC AND ATMOSPHERIC ADMIN',
  u'SEATTLE',
  u'OPERATIONS RESEARCH',
  2015],
 [u'A,MINH D',
  u'14',
  u'GS',
  122571,
  5053,
  u'PATENT AND TRADEMARK OFFICE',
  u'ALEXANDRIA',
  u'PATENT EXAMINING',
  2015],
 [u'A,PHI DIEU TRAN',
  u'14',
  u'GS',
  134063,
  4002,
  u'PATENT AND TRADEMARK OFFICE',
  u'STAFFORD',
  u'PATENT EXAMINING',
  2015],
 [u'A-BOATENG,MARY',
  u'03',
  u'VN',
  90545,
  0,
  u'VETERANS HEALTH ADMINISTRATION',
  u'FAYETTEVILLE',
  u'NURSE',
  2015],
 [u'A-ONIKOYI,OLUREMI',
  u'01',
  u'VN',
  88812,
  0,
  u'VETERANS HEALTH ADMINISTRATION',
  u'PROVIDENCE',
  u'NURSE',
  2015],
 [u'A-ONIKOYI,OLUREMI',

In [2]:
l = [["ALBERT,MILA L","13","GS","$138,914.00","$3,000.00","VETERANS HEALTH ADMINISTRATION","CHICAGO","PHARMACIST","2015"],["ALBERT,NANCY A","06","GS","$43,055.00","$0.00","VETERANS HEALTH ADMINISTRATION","PERRY POINT","MEDICAL SUPPORT ASSISTANCE","2015"],["ALBERT,NICHOLAS","01","VN","$74,336.00","$0.00","VETERANS HEALTH ADMINISTRATION","HOUSTON","NURSE","2015"],["ALBERT,NICHOLAS J","EC","AT","$51,107.00","$0.00","FEDERAL AVIATION ADMINISTRATION","SANTA ROSA","AIR TRAFFIC CONTROL","2015"],["ALBERT,NORMAN NMN","05","WG","$39,674.00","$2,000.00","NATIONAL PARK SERVICE","BLANDING","MASONRY","2015"],["ALBERT,PATRICK M","10","WG","$62,589.00","$0.00","VETERANS HEALTH ADMINISTRATION","SAINT LOUIS","AIR CONDITIONING EQUIPMENT MECHANIC","2015"],["ALBERT,PAUL G","10","WS","$76,885.00","$0.00","VETERANS HEALTH ADMINISTRATION","BALTIMORE","MISC GENERAL MAINTENANCE AND OPERATIONS","2015"],["ALBERT,PAUL S","00","AD","$211,660.00","$0.00","NATIONAL INSTITUTES OF HEALTH","BETHESDA","MATHEMATICAL STATISTICS","2015"],["ALBERT,PETER J","14","GS","$128,244.00","$1,337.00","DEPARTMENT OF ENERGY","PORTLAND","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBERT,PETER S","03","VN","$97,151.00","$650.00","VETERANS HEALTH ADMINISTRATION","BUTLER","NURSE","2015"],["ALBERT,REANNA L.","12","GS","$81,894.00","$1,500.00","INDIAN AFFAIRS","ALBUQUERQUE","EDUCATION PROGRAM","2015"],["ALBERT,REBECCA A","11","GS","$66,969.00","$0.00","VETERANS HEALTH ADMINISTRATION","AKRON","SOCIAL WORK","2015"],["ALBERT,ROBERT C","04","GS","$33,295.00","$65.00","VETERANS HEALTH ADMINISTRATION","MANCHESTER","MEDICAL SUPPORT ASSISTANCE","2015"],["ALBERT,ROBERT J","09","WG","$50,860.00","$0.00","VETERANS HEALTH ADMINISTRATION","MILWAUKEE","CARPENTRY","2015"],["ALBERT,ROBERT L","09","GS","$59,694.00","$0.00","FOREST SERVICE","KALISPELL","FORESTRY TECHNICIAN","2015"],["ALBERT,ROBERTA B","14","GS","$132,368.00","$0.00","NATIONAL INSTITUTES OF HEALTH","BETHESDA","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBERT,ROLAND","13","GS","$116,072.00","$0.00","U.S. MARSHALS SERVICE","NEW HAVEN","ADMINISTRATIVE OFFICER","2015"],["ALBERT,RONALD J","15","GG","$158,700.00","$0.00","NUCLEAR REGULATORY COMMISSION","ROCKVILLE","SECURITY ADMINISTRATION","2015"],["ALBERT,RYAN J.","15","GS","$130,453.00","$3,010.00","ENVIRONMENTAL PROTECTION AGENCY","WASHINGTON","ENVIRONMENTAL PROTECTION SPECIALIST","2015"],["ALBERT,SHARON VANISA","07","GS","$44,844.00","$0.00","RURAL HOUSING SERVICE","SAINT LOUIS","ACCOUNTING TECHNICIAN","2015"],["ALBERT,SHAUN LEE","12","GS","$74,873.00","$0.00","BUREAU OF PRISONS\/FEDERAL PRISON SYSTEM","GREENVILLE","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERT,SHIRLEY B","06","GS","$43,920.00","$0.00","GRAIN INSPEC, PACKERS AND STOCKYARDS ADM","PORT ALLEN","AGRICULTURAL COMMODITY AID","2015"],["ALBERT,SMILEY L JR","07","WG","$43,201.00","$0.00","VETERANS HEALTH ADMINISTRATION","GAINESVILLE","MOTOR VEHICLE OPERATING","2015"],["ALBERT,STEPHEN A","HH","AT","$80,959.00","$0.00","FEDERAL AVIATION ADMINISTRATION","INDIANAPOLIS","AIR TRAFFIC CONTROL","2015"],["ALBERT,STEVEN D","09","GS","$57,320.00","$0.00","CENTERS FOR DISEASE CONTROL AND PREVENTN","ATLANTA","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERT,STEVEN G","14","FG","$131,710.00","$847.00","FEDERAL AVIATION ADMINISTRATION","HILLSBORO","AVIATION SAFETY","2015"],["ALBERT,SUSAN D","11","GS","$76,131.00","$0.00","VETERANS BENEFITS ADMINISTRATION","NEW ORLEANS","VETERANS CLAIMS EXAMINING","2015"],["ALBERT,SUSAN NMN","11","GS","$76,131.00","$0.00","BUREAU OF PRISONS\/FEDERAL PRISON SYSTEM","GREGG TOWNSHIP","PARALEGAL SPECIALIST","2015"],["ALBERT,TANYA","04","GS","$31,409.00","$0.00","INDIAN AFFAIRS","FORT DEFIANCE","FORESTRY TECHNICIAN","2015"],["ALBERT,THOMAS E","00","ST","$173,328.00","$0.00","DOMESTIC NUCLEAR DETECTION OFFICE","WASHINGTON","GENERAL PHYSICAL SCIENCE","2015"],["ALBERT,TRACEY","11","GS","$63,722.00","$0.00","OFC OF ELEM AND SEC ED","WASHINGTON","EDUCATION PROGRAM","2015"],["ALBERT,TRAVER H","KH","AT","$137,163.00","$0.00","FEDERAL AVIATION ADMINISTRATION","HILLIARD","AIR TRAFFIC CONTROL","2015"],["ALBERT,TRAVIS J","07","GS","$40,889.00","$0.00","FOREST SERVICE","CHILOQUIN","BIOLOGICAL SCIENCE TECHNICIAN","2015"],["ALBERT,TYLER J","PH","VM","$205,000.00","$0.00","VETERANS HEALTH ADMINISTRATION","SEATTLE","MEDICAL OFFICER","2015"],["ALBERT,VAL","J","FV","$121,924.00","$0.00","FEDERAL AVIATION ADMINISTRATION","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBERT,VICTORIA L","11","GS","$63,795.00","$84.00","FEDERAL HIGHWAY ADMINISTRATION","VANCOUVER","ENGINEERING TECHNICAL","2015"],["ALBERT,WILLIAM C. JR.","05","GS","$45,190.00","$0.00","FEDERAL STUDENT AID","WASHINGTON","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERT-DEHETRE,PATRICIA","11","GS","$66,370.00","$0.00","VETERANS HEALTH ADMINISTRATION","TOGUS","SOCIAL WORK","2015"],["ALBERT-MAYNOR,SAILOR","06","GS","$47,705.00","$0.00","VETERANS HEALTH ADMINISTRATION","FAYETTEVILLE","PRACTICAL NURSE","2015"],["ALBERT-SAUNDERS,KAMESA","07","GS","$44,613.00","$0.00","VETERANS HEALTH ADMINISTRATION","HOUSTON","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERT-TORRES,JEANNY","02","VN","$60,255.00","$0.00","VETERANS HEALTH ADMINISTRATION","SAN JUAN","NURSE","2015"],["ALBERTA,BRANDON D.","08","WG","$53,907.00","$0.00","NATIONAL PARK SERVICE","YOSEMITE NATL PARK","ENGINEERING EQUIPMENT OPERATING","2015"],["ALBERTA,ELIZABETH M","05","GS","$31,944.00","$0.00","NATIONAL PARK SERVICE","YOSEMITE NATL PARK","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERTA,LAURIE ANN","04","GS","$28,553.00","$0.00","NATIONAL PARK SERVICE","YOSEMITE NATL PARK","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERTA,SETH THOMAS","05","WG","$40,300.00","$0.00","NATIONAL PARK SERVICE","YOSEMITE NATL PARK","ANIMAL CARETAKING","2015"],["ALBERTALLI,BRIAN LOUIS","14","GS","$130,232.00","$7,243.00","PATENT AND TRADEMARK OFFICE","ROSWELL","PATENT EXAMINING","2015"],["ALBERTAZZI,ANNE M","J","SV","$132,308.00","$0.00","TRANSPORTATION SECURITY ADMINISTRATION","ARLINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBERTELLY,MARK","13","GS","$108,507.00","$2,500.00","NAT OCEANIC AND ATMOSPHERIC ADMIN","NORMAN","METEOROLOGY","2015"],["ALBERTI,BARBARA C","03","VN","$96,328.00","$0.00","VETERANS HEALTH ADMINISTRATION","MILWAUKEE","NURSE","2015"],["ALBERTI,BARBARA N.","13","GS","$115,283.00","$700.00","NATIONAL PARK SERVICE","SAN FRANCISCO","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBERTI,CHRISTINE E","11","GS","$68,661.00","$0.00","VETERANS HEALTH ADMINISTRATION","MILWAUKEE","GENERAL ARTS AND INFORMATION","2015"],["ALBERTI,JASON M","09","GS","$50,016.00","$0.00","VETERANS HEALTH ADMINISTRATION","SPOKANE","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERTI,JOEL A","04","WS","$48,210.00","$0.00","BUREAU OF PRISONS\/FEDERAL PRISON SYSTEM","GUAYNABO","LAUNDRY MACHINE OPERATING","2015"],["ALBERTI,MARILYN J","02","VN","$74,362.00","$0.00","VETERANS HEALTH ADMINISTRATION","LEBANON","NURSE","2015"],["ALBERTI,PAUL L","15","GS","$151,496.00","$0.00","DHS HEADQUARTERS","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBERTI,ROSARIO","14","FG","$133,456.00","$518.00","FEDERAL AVIATION ADMINISTRATION","EAST FARMINGDALE","AVIATION SAFETY","2015"],["ALBERTI,SAMUEL E","05","WG","$36,397.00","$0.00","VETERANS HEALTH ADMINISTRATION","WILMINGTON","MATERIALS HANDLER","2015"],["ALBERTIE,HEIDI S","11","GS","$58,562.00","$0.00","VETERANS HEALTH ADMINISTRATION","ORLANDO","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERTINE,ALEXANDER J","03","FP","$87,225.00","$0.00","U.S. AGENCY FOR INTERNATIONAL DEV","CAIRO","ECONOMIST","2015"],["ALBERTINI,BRIAN K","15","GS","$147,288.00","$0.00","NATIONAL INSTITUTES OF HEALTH","BETHESDA","GRANTS MANAGEMENT","2015"],["ALBERTINI,DAVID GREGORY","13","GS","$118,069.00","$153.00","OFFICES, BOARDS AND DIVISIONS","WASHINGTON","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBERTINI,DAWN","13","GS","$96,878.00","$1,000.00","NATIONAL INSTITUTES OF HEALTH","FREDERICK","CONTRACTING","2015"],["ALBERTINI,MARK R","PH","AD","$200,000.00","$0.00","VETERANS HEALTH ADMINISTRATION","MADISON","MEDICAL OFFICER","2015"],["ALBERTO,DENISE K","06","GS","$44,967.00","$0.00","VETERANS HEALTH ADMINISTRATION","MARE ISLAND(NAV SHIPYD)","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERTO,FREDY A","15","GS","$130,453.00","$1,000.00","DEPARTMENT OF ENERGY","WASHINGTON","BUDGET ANALYSIS","2015"],["ALBERTO,LIZETTE I","11","GS","$88,918.00","$0.00","VETERANS HEALTH ADMINISTRATION","FORT WORTH","PHYSICAL THERAPIST","2015"],["ALBERTO,MERLYS","09","IC","$54,576.00","$0.00","FEDERAL EMERGENCY MANAGEMENT AGENCY","NEW YORK","CONTRACTING","2015"],["ALBERTO,NORA G","02","VN","$84,610.00","$78.00","VETERANS HEALTH ADMINISTRATION","HINES","NURSE","2015"],["ALBERTO,THIAGO DA SILVA","01","VN","$55,247.00","$0.00","VETERANS HEALTH ADMINISTRATION","BOISE","NURSE","2015"],["ALBERTO,VALERIE A","12","GS","$84,416.00","$0.00","SOCIAL SECURITY ADMINISTRATION","HOUSTON","GENERAL ATTORNEY","2015"],["ALBERTO,ZORAIDA","12","GS","$83,399.00","$800.00","SOCIAL SECURITY ADMINISTRATION","ONTARIO","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALBERTOLI,RENEE L.","11","GS","$74,971.00","$500.00","NATIONAL PARK SERVICE","PHILADELPHIA","PARK RANGER","2015"],["ALBERTONI,JOHN A","07","GS","$48,800.00","$0.00","U.S. COAST GUARD","ELIZABETH CITY","FIRE PROTECTION AND PREVENTION","2015"],["ALBERTORIO,JUAN R","13","GS","$108,987.00","$0.00","CENTERS FOR DISEASE CONTROL AND PREVENTN","HYATTSVILLE","STATISTICS","2015"],["ALBERTS,BRIAN T","13","GS","$93,851.00","$0.00","FEDERAL TRANSIT ADMINISTRATION","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBERTS,DANIEL B","09","GS","$58,081.00","$0.00","BUREAU OF LAND MANAGEMENT","KANAB","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERTS,DEBRA A","03","VN","$108,007.00","$0.00","VETERANS HEALTH ADMINISTRATION","MINNEAPOLIS","NURSE","2015"],["ALBERTS,ERIC R","13","GS","$83,468.00","$0.00","OFC SEC HEALTH AND HUMAN SERVICES","OCOEE","SUPPLY PROGRAM MANAGEMENT","2015"],["ALBERTS,JANNELL E","09","GS","$48,403.00","$0.00","VETERANS BENEFITS ADMINISTRATION","NEW ORLEANS","VETERANS CLAIMS EXAMINING","2015"],["ALBERTS,JAY L","13","GS","$107,019.00","$0.00","VETERANS HEALTH ADMINISTRATION","CLEVELAND","BIOENGINEERING AND BIOMEDICAL ENGINEERIN","2015"],["ALBERTS,JEFFERY J SR","05","GS","$31,944.00","$0.00","VETERANS HEALTH ADMINISTRATION","LEBANON","PSYCHOLOGY AID AND TECHNICIAN","2015"],["ALBERTS,JESSICA","13","GS","$108,987.00","$0.00","FEDERAL EMERGENCY MANAGEMENT AGENCY","HYATTSVILLE","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERTS,JULI B","07","GS","$50,347.00","$0.00","VETERANS HEALTH ADMINISTRATION","INDIANAPOLIS","HUMAN RESOURCES ASSISTANCE","2015"],["ALBERTS,KEITH ANTHONY","13","GS","$102,932.00","$0.00","FED RETIREMENT THRIFT INVESTMENT BOARD","WASHINGTON","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBERTS,KEITH D.","13","GS","$97,378.00","$0.00","GEOLOGICAL SURVEY","SIOUX FALLS","COMPUTER ENGINEERING","2015"],["ALBERTS,LAWRENCE R.","02","WG","$28,467.00","$0.00","INDIAN AFFAIRS","FORT TOTTEN","CUSTODIAL WORKING","2015"],["ALBERTS,LINDSEY M","12","GS","$91,255.00","$1,377.00","NAT OCEANIC AND ATMOSPHERIC ADMIN","NORMAN","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBERTS,MEGAN C","13","VP","$108,507.00","$0.00","VETERANS HEALTH ADMINISTRATION","EVANSVILLE","OPTOMETRIST","2015"],["ALBERTS,MICHAEL M","J","FV","$117,335.00","$4,000.00","FEDERAL AVIATION ADMINISTRATION","KANSAS CITY","AVIATION SAFETY","2015"],["ALBERTS,MICHELE A","11","GS","$66,585.00","$0.00","DEP ASST SEC FOR INFO AND TECHNOLOGY","SAGINAW","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBERTS,SARA M","11","GS","$70,275.00","$0.00","FOREST SERVICE","TROY","FORESTRY","2015"],["ALBERTS,SCOTT L","13","GS","$95,552.00","$0.00","SOCIAL SECURITY ADMINISTRATION","DENVER","GENERAL ATTORNEY","2015"],["ALBERTS,SETH J","13","GS","$112,926.00","$0.00","LYNDON B. JOHNSON SPACE CENTER","HOUSTON","AEROSPACE ENGINEERING","2015"],["ALBERTS,SETH KIRWIN","06","GS","$36,796.00","$2,272.00","BUREAU OF LAND MANAGEMENT","BOISE","FORESTRY TECHNICIAN","2015"],["ALBERTS,STEPHEN D","12","GS","$70,192.00","$300.00","SOCIAL SECURITY ADMINISTRATION","LEBANON","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALBERTS,TIFFANY M","05","GS","$53,173.00","$0.00","VETERANS HEALTH ADMINISTRATION","MCCLELLAN","PRACTICAL NURSE","2015"],["ALBERTS,TODD C","06","GS","$40,358.00","$760.00","CITIZENSHIP AND IMMIGRATION SERVICES","WILLISTON","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERTSEN,ELIZABETH J","06","GS","$45,107.00","$162.00","BUREAU OF RECLAMATION","GRAND COULEE","SECURITY CLERICAL AND ASSISTANCE","2015"],["ALBERTSEN,MATTHEW D","06","GS","$43,920.00","$0.00","ANIMAL AND PLANT HEALTH INSPECTION SERV","OROVILLE","BIOLOGICAL SCIENCE TECHNICIAN","2015"],["ALBERTSEN,PETER C","00","EH","$65,219.00","$0.00","CENTERS FOR MEDICARE AND MEDICAID SERVIC","WOODLAWN","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERTSON CLARK,SHERRY","00","IM","$66,408.00","$0.00","FEDERAL EMERGENCY MANAGEMENT AGENCY","FORT COLLINS","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERTSON,ALAN P","11","GS","$74,179.00","$1,750.00","BUREAU OF LAND MANAGEMENT","LAKEVIEW","TELECOMMUNICATIONS","2015"],["ALBERTSON,ANN E","12","GS","$99,296.00","$0.00","BROADCASTING BOARD OF GOVERNORS","WASHINGTON","AUDIOVISUAL PRODUCTION","2015"],["ALBERTSON,BEN J","08","GS","$45,284.00","$0.00","FOOD SAFETY AND INSPECTION SERVICE","MONETT","CONSUMER SAFETY INSPECTION","2015"],["ALBERTSON,BERNADETTE G","07","GL","$56,711.00","$0.00","BUREAU OF PRISONS\/FEDERAL PRISON SYSTEM","GREGG TOWNSHIP","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERTSON,CANDICE M","03","WG","$33,642.00","$75.00","VETERANS HEALTH ADMINISTRATION","MURFREESBORO","FOOD SERVICE WORKING","2015"],["ALBERTSON,CINDY W.","14","GS","$121,648.00","$0.00","LANGLEY RESEARCH CENTER","HAMPTON","AEROSPACE ENGINEERING","2015"],["ALBERTSON,CLINT E","11","GS","$64,418.00","$1,000.00","FOREST SERVICE","LAKEVIEW","GEN NATURAL RESOURCES MGT AND BIO SCI","2015"],["ALBERTSON,DENISE J","12","GS","$88,250.00","$160.00","VETERANS HEALTH ADMINISTRATION","PITTSBURGH","HEALTH SYSTEM SPECIALIST","2015"],["ALBERTSON,DONALD JOSEPH","07","GS","$43,526.00","$0.00","FOOD SAFETY AND INSPECTION SERVICE","WATERLOO","FOOD INSPECTION","2015"],["ALBERTSON,DOUGLAS J","12","GS","$70,192.00","$789.00","NAT OCEANIC AND ATMOSPHERIC ADMIN","KANSAS CITY","EQUIPMENT SERVICES","2015"],["ALBERTSON,FERN E","04","GS","$34,265.00","$0.00","BUREAU OF THE CENSUS","MEADVILLE","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERTSON,GARY M.","00","BB","$85,379.00","$0.00","BUREAU OF RECLAMATION","TWIN LAKES","INDUSTRIAL EQUIPMENT MECHANIC","2015"],["ALBERTSON,GEORGE J.","00","BB","$90,158.00","$0.00","BUREAU OF RECLAMATION","TWIN LAKES","INSTRUMENT MECHANIC","2015"],["ALBERTSON,GREG L.","10","WG","$57,643.00","$0.00","BUREAU OF LAND MANAGEMENT","LAKEVIEW","ENGINEERING EQUIPMENT OPERATING","2015"],["ALBERTSON,HEIDI LYNN","11","GS","$64,418.00","$500.00","BUREAU OF LAND MANAGEMENT","ALTURAS","RANGELAND MANAGEMENT","2015"],["ALBERTSON,JANALEE","07","GS","$50,119.00","$0.00","FOREST SERVICE","ASHTON","SUPPORT SERVICES ADMINISTRATION","2015"],["ALBERTSON,JANELLA M","06","GS","$46,294.00","$0.00","SOCIAL SECURITY ADMINISTRATION","WILKES BARRE","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERTSON,JOANNE","03","VN","$111,789.00","$0.00","VETERANS HEALTH ADMINISTRATION","BAY PINES","NURSE","2015"],["ALBERTSON,JOHN K","14","GS","$115,072.00","$350.00","DEPARTMENT OF ENERGY","AIKEN","GENERAL ENGINEERING","2015"],["ALBERTSON,JOHN P","07","GS","$43,012.00","$0.00","VETERANS BENEFITS ADMINISTRATION","DETROIT","VETERANS CLAIMS EXAMINING","2015"],["ALBERTSON,JOY D.","12","GS","$99,722.00","$0.00","U.S. FISH AND WILDLIFE SERVICE","FREMONT","WILDLIFE BIOLOGY","2015"],["ALBERTSON,JUDY L","13","GS","$91,814.00","$0.00","VETERANS HEALTH ADMINISTRATION","BAY PINES","ACCOUNTING","2015"],["ALBERTSON,KAREN S","12","GS","$91,255.00","$300.00","VETERANS BENEFITS ADMINISTRATION","JACKSONVILLE","SOCIAL SCIENCE","2015"],["ALBERTSON,KENDRA S","13","GS","$97,378.00","$0.00","CITIZENSHIP AND IMMIGRATION SERVICES","WILLISTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBERTSON,KRISTIN H","11","GS","$74,179.00","$326.00","SOCIAL SECURITY ADMINISTRATION","ROME","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALBERTSON,LARRY R","10","WG","$52,238.00","$250.00","U.S. COAST GUARD","ELIZABETH CITY","MAINTENANCE MECHANIC","2015"],["ALBERTSON,LAVONNE E","02","AD","$93,974.00","$0.00","VETERANS HEALTH ADMINISTRATION","PORTLAND","NURSE","2015"],["ALBERTSON,LEIF E","13","GS","$91,814.00","$0.00","OFFICE OF GSA IT","PORTLAND","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBERTSON,MATTHEW G","12","GS","$77,213.00","$0.00","CUSTOMS AND BORDER PROTECTION","LAREDO","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBERTSON,MERRILL JAMES","05","GS","$35,427.00","$200.00","VETERANS HEALTH ADMINISTRATION","DENVER","SECURITY CLERICAL AND ASSISTANCE","2015"],["ALBERTSON,NATHAN M","08","GS","$51,187.00","$0.00","INDIAN HEALTH SERVICE","BELCOURT","DIAGNOSTIC RADIOLOGIC TECHNOLOGIST","2015"],["ALBERTSON,PATRICIA L","08","GS","$48,206.00","$0.00","VETERANS HEALTH ADMINISTRATION","FAYETTEVILLE","PATHOLOGY TECHNICIAN","2015"],["ALBERTSON,PAUL D","00","IM","$50,443.00","$0.00","FEDERAL EMERGENCY MANAGEMENT AGENCY","GOODYEAR","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERTSON,PAULA C","15","GS","$115,182.00","$0.00","BUREAU OF INTERNATIONAL LABOR AFFAIRS","DHAKA","INTERNATIONAL RELATIONS","2015"],["ALBERTSON,PETER A","12","GS","$76,322.00","$0.00","VETERANS HEALTH ADMINISTRATION","ASPINWALL","HOSPITAL HOUSEKEEPING MANAGEMENT","2015"],["ALBERTSON,RANDAL","14","GS","$139,163.00","$0.00","NEIL A. ARMSTRONG FLIGHT RESEARCH CENTER","EDWARDS AFB","PROGRAM MANAGEMENT","2015"],["ALBERTSON,RICHARD A","11","GS","$70,818.00","$0.00","VETERANS BENEFITS ADMINISTRATION","SEATTLE","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBERTSON,RONALD A","11","GS","$68,322.00","$280.00","DEPARTMENT OF ENERGY","WATERTOWN","FACILITY OPERATIONS SERVICES","2015"],["ALBERTSON,SARAH B.","14","SK","$162,970.00","$1,000.00","SECURITIES AND EXCHANGE COMMISSION","WASHINGTON","GENERAL ATTORNEY","2015"],["ALBERTSON,SEAN TOOLE","04","GS","$28,553.00","$0.00","BUREAU OF LAND MANAGEMENT","BOISE","ADMIN AND OFFICE SUPPORT STUDENT TRAINEE","2015"],["ALBERTSON,SUMMER L","02","VN","$63,168.00","$78.00","VETERANS HEALTH ADMINISTRATION","FORT SMITH","NURSE","2015"],["ALBERTSON,THOMAS L","09","WS","$71,835.00","$0.00","FOREST SERVICE","LAKEVIEW","ENGINEERING EQUIPMENT OPERATING","2015"],["ALBERTSON,TIMOTHY E","PH","AD","$234,619.00","$0.00","VETERANS HEALTH ADMINISTRATION","MATHER AFB","MEDICAL OFFICER","2015"],["ALBERTUS,PAUL S","02","ET","$145,000.00","$0.00","DEPARTMENT OF ENERGY","WASHINGTON","GENERAL PHYSICAL SCIENCE","2015"],["ALBERTY,ALITA M","03","GS","$25,434.00","$0.00","NAT ARCHIVES AND RECORDS ADMINISTRATION","SPANISH LAKE","DATA TRANSCRIBER","2015"],["ALBERTY,CAROLINE A","02","VN","$85,845.00","$0.00","VETERANS HEALTH ADMINISTRATION","MILWAUKEE","NURSE","2015"],["ALBERTY,JAMES E JR","02","WG","$26,463.00","$250.00","VETERANS HEALTH ADMINISTRATION","MUSKOGEE","CUSTODIAL WORKING","2015"],["ALBERTY,KATHLEEN D","05","GS","$37,270.00","$0.00","VETERANS HEALTH ADMINISTRATION","MUSKOGEE","NURSING ASSISTANT","2015"],["ALBERTY,LINDA B","15","GS","$158,700.00","$2,000.00","EXECUTIVE OFFICE FOR IMMIGRATION REVIEW","FALLS CHURCH","GENERAL ATTORNEY","2015"],["ALBERTY,MARIA JEWEL","13","GS","$111,989.00","$0.00","AMES RESEARCH CENTER","MOFFETT FIELD","CONTRACTING","2015"],["ALBERTY,MARY E","11","GS","$66,370.00","$0.00","VETERANS HEALTH ADMINISTRATION","WHITE RIVER JCTN","DIETITIAN AND NUTRITIONIST","2015"],["ALBERTY,SHANTON L","09","GS","$53,242.00","$0.00","VETERANS BENEFITS ADMINISTRATION","MUSKOGEE","VETERANS CLAIMS EXAMINING","2015"],["ALBERY,BRENDA M.","12","GS","$75,333.00","$0.00","BUREAU OF RECLAMATION","LAKEWOOD","ACCOUNTING","2015"],["ALBERY,SANDRA M","05","GS","$38,335.00","$50.00","VETERANS HEALTH ADMINISTRATION","SALEM","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBERY,TRACEY J","01","VN","$55,450.00","$0.00","VETERANS HEALTH ADMINISTRATION","ORLANDO","NURSE","2015"],["ALBETE,CHARLENE S","05","GS","$31,944.00","$0.00","VETERANS HEALTH ADMINISTRATION","LAS VEGAS","MEDICAL RECORDS TECHNICIAN","2015"],["ALBETKOVA,ADILYA A","00","RG","$90,126.00","$0.00","CENTERS FOR DISEASE CONTROL AND PREVENTN","ATLANTA","GENERAL HEALTH SCIENCE","2015"],["ALBEZA,DARLENE P","02","VN","$86,876.00","$0.00","VETERANS HEALTH ADMINISTRATION","CHICAGO","NURSE","2015"],["ALBEZA,MICHAEL S","02","VN","$86,876.00","$78.00","VETERANS HEALTH ADMINISTRATION","HINES","NURSE","2015"],["ALBEZA,ROSVI B","02","VN","$87,386.00","$0.00","VETERANS HEALTH ADMINISTRATION","LAS VEGAS","NURSE","2015"],["ALBI,GIOIA","J","FV","$123,767.00","$0.00","FEDERAL AVIATION ADMINISTRATION","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBI,STEPHEN M","13","GS","$122,346.00","$0.00","CUSTOMS AND BORDER PROTECTION","NEW YORK","GENERAL BUSINESS AND INDUSTRY","2015"],["ALBIBI,RAJAI","12","GS","$91,255.00","$0.00","FOOD SAFETY AND INSPECTION SERVICE","SAINT LOUIS","CHEMISTRY","2015"],["ALBIDRESS,RICHARD JR","12","IC","$82,387.00","$0.00","FEDERAL EMERGENCY MANAGEMENT AGENCY","BOTHELL","CONTRACTING","2015"],["ALBIEZ,JEANNETTE","13","GS","$115,283.00","$1,450.00","AMES RESEARCH CENTER","MOFFETT FIELD","CONTRACTING","2015"],["ALBIN,ANN HARRIS","15","GS","$158,700.00","$0.00","DEPARTMENTAL OFFICES","WASHINGTON","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBIN,ASHLEY E","11","GS","$76,131.00","$0.00","VETERANS HEALTH ADMINISTRATION","TUCSON","DIETITIAN AND NUTRITIONIST","2015"],["ALBIN,BRAD R","07","GS","$42,207.00","$350.00","VETERANS BENEFITS ADMINISTRATION","MUSKOGEE","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBIN,BRANDY E","11","GS","$62,466.00","$0.00","BUREAU OF LAND MANAGEMENT","ROSEBURG","CIVIL ENGINEERING","2015"],["ALBIN,DAVID J","11","GS","$68,603.00","$0.00","DEP ASST SEC FOR INFO AND TECHNOLOGY","OMAHA","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBIN,EAN J","00","BB","$88,656.00","$1,037.00","DEPARTMENT OF ENERGY","REDMOND","HIGH VOLTAGE ELECTRICIAN","2015"],["ALBIN,FRANKLIN E","H","FV","$95,719.00","$3,520.00","FEDERAL AVIATION ADMINISTRATION","OLATHE","TRANSPORTATION SPECIALIST","2015"],["ALBIN,JENIFER N","11","GS","$66,370.00","$0.00","VETERANS BENEFITS ADMINISTRATION","MUSKOGEE","VETERANS CLAIMS EXAMINING","2015"],["ALBIN,JENNIFER J","12","GS","$86,006.00","$0.00","NAT ARCHIVES AND RECORDS ADMINISTRATION","PERRIS","ARCHIVIST","2015"],["ALBIN,JOSEPH F","11","CU","$98,725.00","$750.00","NATIONAL CREDIT UNION ADMINISTRATION","CLEVELAND","CREDIT UNION EXAMINER","2015"],["ALBIN,MICHAEL E","12","GS","$86,574.00","$835.00","FARM SERVICE AGENCY","CHARLESTON","LOAN SPECIALIST","2015"],["ALBIN,RICHARD B","13","GS","$115,778.00","$900.00","FEDERAL HIGHWAY ADMINISTRATION","OLYMPIA","CIVIL ENGINEERING","2015"],["ALBIN,ROGER L","PH","AD","$170,619.00","$0.00","VETERANS HEALTH ADMINISTRATION","ANN ARBOR","MEDICAL OFFICER","2015"],["ALBIN,SIGRID S","06","GS","$42,732.00","$0.00","VETERANS HEALTH ADMINISTRATION","BAY PINES","SUPPLY CLERICAL AND TECHNICIAN","2015"],["ALBIN,TIFANY N","13","GS","$91,814.00","$0.00","DEPARTMENT OF ENERGY","RICHLAND","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBINA,DENISE","09","GS","$50,578.00","$0.00","CENTERS FOR DISEASE CONTROL AND PREVENTN","ATLANTA","PUBLIC HEALTH PROGRAM SPECIALIST","2015"],["ALBING,DANIEL M","KH","AT","$127,377.00","$100.00","FEDERAL AVIATION ADMINISTRATION","FARMINGTON","AIR TRAFFIC CONTROL","2015"],["ALBINO,ELIZABETH A","01","VN","$74,116.00","$0.00","VETERANS HEALTH ADMINISTRATION","PROVIDENCE","NURSE","2015"],["ALBINO,JAMES","15","GS","$130,453.00","$0.00","NAT PROTECTION AND PROGRAMS DIRECTORATE","ARLINGTON","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBINO,JAMES","00","ES","$140,000.00","$0.00","DEP ASST SEC INTERGOVERNMENTAL AFFAIRS","WASHINGTON","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBINO,LUIS A","02","WG","$35,416.00","$0.00","VETERANS HEALTH ADMINISTRATION","WILKES BARRE","CUSTODIAL WORKING","2015"],["ALBINO,NILSA J","09","GS","$62,920.00","$0.00","VETERANS HEALTH ADMINISTRATION","DAYTONA BEACH","MEDICAL TECHNOLOGIST","2015"],["ALBINO,WISELL","12","GS","$96,555.00","$0.00","VETERANS HEALTH ADMINISTRATION","MIAMI","GENERAL ENGINEERING","2015"],["ALBINO-PEREZ,CARMEN","13","GS","$97,378.00","$500.00","VETERANS BENEFITS ADMINISTRATION","SAN JUAN","VETERANS CLAIMS EXAMINING","2015"],["ALBINOLA,KRISTINA L","08","GS","$45,284.00","$0.00","VETERANS HEALTH ADMINISTRATION","SPOKANE","DIAGNOSTIC RADIOLOGIC TECHNOLOGIST","2015"],["ALBINS,LISA M","13","VN","$83,468.00","$0.00","VETERANS HEALTH ADMINISTRATION","COLUMBIA","PHYSICIAN ASSISTANT","2015"],["ALBINSON,GRACE E.","14","GS","$114,480.00","$0.00","OFFICES, BOARDS AND DIVISIONS","WASHINGTON","GENERAL ATTORNEY","2015"],["ALBISUREZ,EDGAR G","12","GS","$79,293.00","$0.00","FEDERAL MOTOR CARRIER SAFETY ADMIN","BURLINGTON","MOTOR CARRIER SAFETY","2015"],["ALBITO,ANTHONY L","PH","VM","$210,000.00","$1,000.00","VETERANS HEALTH ADMINISTRATION","SAGINAW","MEDICAL OFFICER","2015"],["ALBITRE,GARY J","10","WG","$71,250.00","$0.00","VETERANS HEALTH ADMINISTRATION","SAN FRANCISCO","GENERAL EQUIPMENT MECHANIC","2015"],["ALBITRE,JAMES R","12","GS","$96,336.00","$0.00","INDIAN HEALTH SERVICE","ELKO","PHYSICIAN ASSISTANT","2015"],["ALBIZU,RAMON D.","13","GS","$102,109.00","$0.00","ENVIRONMENTAL PROTECTION AGENCY","SAN FRANCISCO","ENVIRONMENTAL PROTECTION SPECIALIST","2015"],["ALBL,BERNHARD M. H.","13","GS","$97,378.00","$300.00","BUREAU OF RECLAMATION","BOISE","SECURITY ADMINISTRATION","2015"],["ALBLINGER,CARRIE B","13","GS","$99,905.00","$0.00","BUREAU OF THE CENSUS","SUITLAND","GEOGRAPHY","2015"],["ALBO,DANIEL","LH","AT","$137,740.00","$100.00","FEDERAL AVIATION ADMINISTRATION","SAN DIEGO","AIR TRAFFIC CONTROL","2015"],["ALBO,DAVID MICHAEL","05","GS","$37,029.00","$0.00","FOREST SERVICE","PHOENIX","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBO,MICHAEL E","PH","AD","$299,098.00","$0.00","VETERANS HEALTH ADMINISTRATION","SAN DIEGO","MEDICAL OFFICER","2015"],["ALBO,PAULA MARIE","06","GS","$37,983.00","$0.00","VETERANS HEALTH ADMINISTRATION","PUEBLO","MEDICAL SUPPORT ASSISTANCE","2015"],["ALBO,RHONDA L","12","GS","$81,894.00","$0.00","VETERANS HEALTH ADMINISTRATION","RENO","ADMINISTRATIVE OFFICER","2015"],["ALBO,THOMAS L","07","GS","$53,789.00","$0.00","OFFICE OF THE SECRETARY OF THE INTERIOR","LAKEWOOD","ACCOUNTING TECHNICIAN","2015"],["ALBOHER,STACY A","4A","MC","$98,299.00","$0.00","MILLENNIUM CHALLENGE CORPORATION","WASHINGTON","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBOHN,CHARLES D","09","GS","$48,403.00","$0.00","VETERANS BENEFITS ADMINISTRATION","WINSTON SALEM","VETERANS CLAIMS EXAMINING","2015"],["ALBOLD,JASON T","KH","AT","$146,628.00","$0.00","FEDERAL AVIATION ADMINISTRATION","OLATHE","AIR TRAFFIC CONTROL","2015"],["ALBON,AMBER S","06","GS","$35,609.00","$0.00","VETERANS HEALTH ADMINISTRATION","CLEARLAKE","MEDICAL SUPPORT ASSISTANCE","2015"],["ALBOR,HEIDI G","05","GS","$35,005.00","$0.00","SOCIAL SECURITY ADMINISTRATION","CHICAGO","CONTACT REPRESENTATIVE","2015"],["ALBORES,EMIR S","14","GS","$114,480.00","$0.00","HEADQUARTERS, NASA","WASHINGTON","BUDGET ANALYSIS","2015"],["ALBORES,RICHARD L.","15","GS","$158,700.00","$2,800.00","ENVIRONMENTAL PROTECTION AGENCY","WASHINGTON","GENERAL ATTORNEY","2015"],["ALBORN,HANS T","14","GS","$101,921.00","$0.00","AGRICULTURAL RESEARCH SERVICE","GAINESVILLE","CHEMISTRY","2015"],["ALBORNOZ,LAURA M","00","ES","$176,337.00","$0.00","MERIT SYSTEMS PROTECTION BOARD","DALLAS","GENERAL ATTORNEY","2015"],["ALBORS-SANCHEZ,JOAN","05","AD","$52,638.00","$0.00","VETERANS HEALTH ADMINISTRATION","SAN JUAN","MEDICAL OFFICER","2015"],["ALBOUARI,MARIA E","14","GS","$125,213.00","$0.00","CITIZENSHIP AND IMMIGRATION SERVICES","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBOUM,JONATHAN","00","ES","$183,300.00","$0.00","OFFICE OF THE CHIEF INFORMATION OFFICER","WASHINGTON","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBOUYEH,DARA","J","FV","$140,779.00","$0.00","FEDERAL AVIATION ADMINISTRATION","","AEROSPACE ENGINEERING","2015"],["ALBRACHT,DAVID","PH","VM","$165,653.00","$0.00","VETERANS HEALTH ADMINISTRATION","DALLAS","MEDICAL OFFICER","2015"],["ALBRACHT,GLEN E","15","GS","$155,838.00","$0.00","VETERANS HEALTH ADMINISTRATION","SALISBURY","PHARMACIST","2015"],["ALBRACHT,JAMES T","11","GS","$75,004.00","$0.00","VETERANS HEALTH ADMINISTRATION","FORT THOMAS","VOCATIONAL REHABILITATION","2015"],["ALBRACHT,KIMBERLY ANN","11","GS","$66,370.00","$0.00","FOREST SERVICE","BOISE","TELECOMMUNICATIONS","2015"],["ALBRACHT,ROBERT M","13","GS","$118,069.00","$459.00","ASST SEC FOR PUB AND INDIAN HOUSING","WASHINGTON","GENERAL ENGINEERING","2015"],["ALBRECHT,AILA P","14","GS","$121,635.00","$0.00","FOOD AND DRUG ADMINISTRATION","ROCKVILLE","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBRECHT,ALAN R","12","GS","$77,213.00","$0.00","FOOD AND DRUG ADMINISTRATION","PEMBINA","CONSUMER SAFETY","2015"],["ALBRECHT,AMANDA B","05","GS","$39,178.00","$306.00","FOREST SERVICE","RAMONA","FORESTRY TECHNICIAN","2015"],["ALBRECHT,AMANDA LYNN","05","GS","$33,677.00","$0.00","VETERANS HEALTH ADMINISTRATION","RICHMOND","MEDICAL SUPPORT ASSISTANCE","2015"],["ALBRECHT,ANDREW S","06","GS","$35,609.00","$0.00","AGRICULTURAL RESEARCH SERVICE","CORVALLIS","BIOLOGICAL SCIENCE TECHNICIAN","2015"],["ALBRECHT,ARLENE L","05","GS","$45,179.00","$0.00","VETERANS BENEFITS ADMINISTRATION","SAN DIEGO","CLAIMS ASSISTANCE AND EXAMINING","2015"],["ALBRECHT,AUDREY ANNA","07","GS","$39,570.00","$0.00","GEOLOGICAL SURVEY","JAMESTOWN","BIOLOGICAL SCIENCE TECHNICIAN","2015"],["ALBRECHT,BRIAN J","13","GS","$91,814.00","$0.00","OCCUPATIONAL SAFETY AND HEALTH ADM","SALT LAKE CITY","CHEMISTRY","2015"],["ALBRECHT,CAROLINE M","12","GS","$116,836.00","$0.00","VETERANS HEALTH ADMINISTRATION","MINNEAPOLIS","PHARMACIST","2015"],["ALBRECHT,CATHERINE S","13","GS","$100,161.00","$1,337.00","DEPARTMENT OF ENERGY","REDMOND","REALTY","2015"],["ALBRECHT,CHRISTOPHER J","11","GS","$79,145.00","$500.00","VETERANS HEALTH ADMINISTRATION","CLEVELAND","GENERAL HEALTH SCIENCE","2015"],["ALBRECHT,CHRISTOPHER S","K","FV","$123,065.00","$1,500.00","FEDERAL AVIATION ADMINISTRATION","CORAOPOLIS","AVIATION SAFETY","2015"],["ALBRECHT,DANIEL J","13","GS","$90,823.00","$400.00","CENTERS FOR MEDICARE AND MEDICAID SERVIC","WOODLAWN","BUDGET ANALYSIS","2015"],["ALBRECHT,DENISE K","06","GS","$46,294.00","$0.00","VETERANS HEALTH ADMINISTRATION","NORFOLK","PRACTICAL NURSE","2015"],["ALBRECHT,DESIREE L","09","GS","$51,646.00","$0.00","SMALL BUSINESS ADMINISTRATION","SEATTLE","GENERAL BUSINESS AND INDUSTRY","2015"],["ALBRECHT,DOROTHY K","03","VN","$94,689.00","$0.00","VETERANS HEALTH ADMINISTRATION","SALEM","NURSE","2015"],["ALBRECHT,DUDLEY O","04","GS","$39,736.00","$0.00","BUREAU OF LAND MANAGEMENT","SACRAMENTO","EQUIPMENT OPERATOR","2015"],["ALBRECHT,EDWARD J","13","GS","$118,620.00","$0.00","CENTERS FOR MEDICARE AND MEDICAID SERVIC","BOSTON","HEALTH INSURANCE ADMINISTRATION","2015"],["ALBRECHT,ERIC J","15","GS","$151,496.00","$0.00","PUBLIC BUILDINGS SERVICE","WASHINGTON","CIVIL ENGINEERING","2015"],["ALBRECHT,ERIC N","H","FV","$63,021.00","$1,500.00","FEDERAL AVIATION ADMINISTRATION","HILLIARD","AIR TRAFFIC ASSISTANCE","2015"],["ALBRECHT,ERNEST L","01","VN","$50,524.00","$0.00","VETERANS HEALTH ADMINISTRATION","WACO","NURSE","2015"],["ALBRECHT,FRANCINE E","05","GS","$34,085.00","$0.00","VETERANS HEALTH ADMINISTRATION","SEATTLE","MEDICAL SUPPORT ASSISTANCE","2015"],["ALBRECHT,FRANK A.","03","WG","$34,519.00","$0.00","NATIONAL PARK SERVICE","YELLOWSTONE PARK","LABORING","2015"],["ALBRECHT,FREDERIC P","IH","AT","$127,776.00","$0.00","FEDERAL AVIATION ADMINISTRATION","MILWAUKEE","AIR TRAFFIC CONTROL","2015"],["ALBRECHT,HEATH R","12","GS","$72,533.00","$0.00","NAT HIGHWAY TRAFFIC SAFETY ADM","EAST LIBERTY","ELECTRONICS ENGINEERING","2015"],["ALBRECHT,JAMES CARL","10","WG","$58,540.00","$0.00","NATIONAL INSTITUTES OF HEALTH","HAMILTON","MAINTENANCE MECHANIC","2015"],["ALBRECHT,JAMES P II","12","FG","$76,667.00","$0.00","FEDERAL AVIATION ADMINISTRATION","JUNEAU","AVIATION SAFETY","2015"],["ALBRECHT,JAMIE L","09","GS","$57,302.00","$0.00","OFC OF CITIZEN SVS AND INNOVATIVE TECH","SAN FRANCISCO","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBRECHT,JAY A","13","GS","$115,778.00","$1,600.00","NAT OCEANIC AND ATMOSPHERIC ADMIN","SEATTLE","METEOROLOGY","2015"],["ALBRECHT,JEFFREY H","PH","VM","$285,000.00","$0.00","VETERANS HEALTH ADMINISTRATION","MINNEAPOLIS","MEDICAL OFFICER","2015"],["ALBRECHT,JESSICA A","12","GS","$89,527.00","$650.00","SOCIAL SECURITY ADMINISTRATION","BOSTON","GENERAL ATTORNEY","2015"],["ALBRECHT,JOEL A","08","WS","$64,050.00","$0.00","BUREAU OF PRISONS\/FEDERAL PRISON SYSTEM","SHERIDAN","COOKING","2015"],["ALBRECHT,JOSHUA D","12","FG","$71,551.00","$256.00","FEDERAL AVIATION ADMINISTRATION","BRENTWOOD","AVIATION SAFETY","2015"],["ALBRECHT,KEVIN D","11","GS","$70,275.00","$0.00","FOREST SERVICE","FERRON","WILDLIFE BIOLOGY","2015"],["ALBRECHT,KEVIN J","11","GS","$71,818.00","$0.00","VETERANS HEALTH ADMINISTRATION","SALT LAKE CITY","DIAGNOSTIC RADIOLOGIC TECHNOLOGIST","2015"],["ALBRECHT,LAURIE J","05","GS","$34,085.00","$0.00","SOCIAL SECURITY ADMINISTRATION","AUBURN","CONTACT REPRESENTATIVE","2015"],["ALBRECHT,LILA M","12","GS","$77,213.00","$0.00","VETERANS BENEFITS ADMINISTRATION","DES MOINES","VETERANS CLAIMS EXAMINING","2015"],["ALBRECHT,LORI","12","CG","$130,069.00","$2,142.00","FEDERAL DEPOSIT INSURANCE CORPORATION","BLUE BELL","FINANCIAL INSTITUTION EXAMINING","2015"],["ALBRECHT,LYNN M","08","GS","$58,940.00","$0.00","VETERANS HEALTH ADMINISTRATION","MILWAUKEE","MEDICAL RECORDS TECHNICIAN","2015"],["ALBRECHT,MARICELA M","02","WS","$42,491.00","$0.00","VETERANS HEALTH ADMINISTRATION","KERRVILLE","CUSTODIAL WORKING","2015"],["ALBRECHT,MARK J","12","GS","$91,255.00","$0.00","MINE SAFETY AND HEALTH ADMINISTRATION","GILLETTE","MINE SAFETY AND HEALTH INSPECTION SERIES","2015"],["ALBRECHT,MARK T","14","GS","$118,057.00","$0.00","OFC SEC HEALTH AND HUMAN SERVICES","WASHINGTON","GENERAL HEALTH SCIENCE","2015"],["ALBRECHT,MATTHEW J","KF","AT","$89,152.00","$0.00","FEDERAL AVIATION ADMINISTRATION","HOUSTON","AIR TRAFFIC CONTROL","2015"],["ALBRECHT,NICHOLAS C","09","GS","$48,403.00","$0.00","U.S. FISH AND WILDLIFE SERVICE","ENTIAT","FISH BIOLOGY","2015"],["ALBRECHT,PATRICIA L","09","GS","$66,668.00","$0.00","VETERANS HEALTH ADMINISTRATION","MINNEAPOLIS","MEDICAL RECORDS TECHNICIAN","2015"],["ALBRECHT,PETER M","11","GS","$80,812.00","$1,179.00","PATENT AND TRADEMARK OFFICE","ALEXANDRIA","PATENT EXAMINING","2015"],["ALBRECHT,RANDALL J","13","GS","$122,346.00","$310.00","SOCIAL SECURITY ADMINISTRATION","JERICHO","GENERAL ATTORNEY","2015"],["ALBRECHT,RENATA","15","GP","$248,112.00","$0.00","FOOD AND DRUG ADMINISTRATION","ROCKVILLE","MEDICAL OFFICER","2015"],["ALBRECHT,RICHARD L","13","GS","$112,014.00","$0.00","OFFICE OF THE INSPECTOR GENERAL","WASHINGTON","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBRECHT,ROBERT D","11","WG","$58,332.00","$0.00","VETERANS HEALTH ADMINISTRATION","OKLAHOMA CITY","SHEET METAL MECHANIC","2015"],["ALBRECHT,RONALD F II","PH","VM","$350,000.00","$0.00","VETERANS HEALTH ADMINISTRATION","CHICAGO","MEDICAL OFFICER","2015"],["ALBRECHT,SANDY","06","GS","$43,920.00","$0.00","NATURAL RESOURCES CONSERVATION SERVICE","RICHFIELD","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBRECHT,SARAH BAYKO","14","GS","$114,480.00","$550.00","OFC SEC HEALTH AND HUMAN SERVICES","WASHINGTON","CIVIL RIGHTS ANALYSIS","2015"],["ALBRECHT,SCOTT ALAN","06","GS","$43,313.00","$1,200.00","FOREST SERVICE","FORT COLLINS","FORESTRY TECHNICIAN","2015"],["ALBRECHT,SCOTT GARRETT","11","GS","$63,722.00","$0.00","BUREAU OF THE CENSUS","SUITLAND","STATISTICS","2015"],["ALBRECHT,SHEA W","13","GS","$85,084.00","$150.00","OFFICE OF PERSONNEL MANAGEMENT","BOYERS","HUMAN RESOURCES MANAGEMENT","2015"],["ALBRECHT,SUSAN M","13","GS","$118,620.00","$500.00","CENTERS FOR MEDICARE AND MEDICAID SERVIC","BOSTON","GENERAL HEALTH SCIENCE","2015"],["ALBRECHT,TERENCE W","13","GS","$115,041.00","$0.00","FOREIGN AGRICULTURAL SERVICE","WASHINGTON","INTERNATIONAL RELATIONS","2015"],["ALBRECHT,TRACY","11","GS","$80,452.00","$0.00","BUREAU OF LAND MANAGEMENT","PALM DESERT","GENERAL ARTS AND INFORMATION","2015"],["ALBRECHT,TYSON A","11","GS","$58,562.00","$0.00","FOREST SERVICE","POMEROY","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBRECHT,UTE","11","GS","$72,227.00","$0.00","AGRICULTURAL RESEARCH SERVICE","FT PIERCE","GEN NATURAL RESOURCES MGT AND BIO SCI","2015"],["ALBRECHT,VERTIE A","13","GS","$108,987.00","$0.00","CUSTOMS AND BORDER PROTECTION","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBRECHT,WAYNE J","12","GS","$125,036.00","$0.00","VETERANS HEALTH ADMINISTRATION","PHILADELPHIA","PHARMACIST","2015"],["ALBRECHT,WILLIAM C","09","GS","$58,081.00","$0.00","VETERANS HEALTH ADMINISTRATION","FARGO","MEDICAL TECHNOLOGIST","2015"],["ALBRECHT,WILLIAM J","08","WL","$55,285.00","$1,008.00","VETERANS HEALTH ADMINISTRATION","MILWAUKEE","MOTOR VEHICLE OPERATING","2015"],["ALBRESKI,SHANNON D","08","GS","$48,206.00","$0.00","SOCIAL SECURITY ADMINISTRATION","ALBUQUERQUE","CONTACT REPRESENTATIVE","2015"],["ALBRIGHT BURLEY,ANGELA","12","GS","$81,894.00","$0.00","SOCIAL SECURITY ADMINISTRATION","TUCSON","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALBRIGHT GAYMON,ANITA K","05","NB","$138,945.00","$0.00","OFC OF COMPTROLLER OF CURRENCY","MIAMI","FINANCIAL INSTITUTION EXAMINING","2015"],["ALBRIGHT,AARON K","15","GS","$130,453.00","$1,000.00","CENTERS FOR MEDICARE AND MEDICAID SERVIC","WASHINGTON","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBRIGHT,ADA B","14","GS","$121,635.00","$0.00","DHS HEADQUARTERS","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBRIGHT,ALESA D","09","GS","$66,710.00","$0.00","SOCIAL SECURITY ADMINISTRATION","BALTIMORE","GENERAL LEGAL AND KINDRED ADMINISTRATION","2015"],["ALBRIGHT,ANDREW","03","GS","$25,434.00","$0.00","FOREST SERVICE","JOHN DAY","FORESTRY TECHNICIAN","2015"],["ALBRIGHT,ANDREW M","13","GS","$103,524.00","$250.00","OFC SEC HEALTH AND HUMAN SERVICES","NEW YORK","GENERAL HEALTH SCIENCE","2015"],["ALBRIGHT,ANN L","00","RF","$206,187.00","$0.00","CENTERS FOR DISEASE CONTROL AND PREVENTN","ATLANTA","GENERAL HEALTH SCIENCE","2015"],["ALBRIGHT,ASHLEY L","02","VC","$20,787.00","$0.00","VETERANS HEALTH ADMINISTRATION","ASPINWALL","SALES STORE CLERICAL","2015"],["ALBRIGHT,BRUCE S","15","GS","$158,700.00","$0.00","OFFICE OF GENERAL COUNSEL","WASHINGTON","GENERAL ATTORNEY","2015"],["ALBRIGHT,BRYAN M","02","WG","$33,037.00","$0.00","VETERANS HEALTH ADMINISTRATION","LEBANON","LAUNDRY WORKING","2015"],["ALBRIGHT,CAROL A","05","WG","$45,643.00","$0.00","VETERANS HEALTH ADMINISTRATION","INDIANAPOLIS","PRESCRIPTION EYEGLASS MAKING","2015"],["ALBRIGHT,CHAD E","12","GS","$70,512.00","$0.00","CUSTOMS AND BORDER PROTECTION","INDIANAPOLIS","HUMAN RESOURCES MANAGEMENT","2015"],["ALBRIGHT,CHRIS A","14","GS","$111,784.00","$0.00","VETERANS HEALTH ADMINISTRATION","HUNTINGTON","GENERAL ENGINEERING","2015"],["ALBRIGHT,CONCHETTA J","12","GS","$87,367.00","$0.00","VETERANS BENEFITS ADMINISTRATION","PHILADELPHIA","VETERANS CLAIMS EXAMINING","2015"],["ALBRIGHT,DAVID S","04","GS","$31,409.00","$0.00","BUREAU OF THE CENSUS","ROCHESTER","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBRIGHT,DAVID S.","14","GS","$147,907.00","$1,375.00","ENVIRONMENTAL PROTECTION AGENCY","SAN FRANCISCO","ENVIRONMENTAL PROTECTION SPECIALIST","2015"],["ALBRIGHT,DEBORAH W","08","WG","$54,512.00","$0.00","VETERANS HEALTH ADMINISTRATION","COATESVILLE","AIR CONDITIONING EQUIPMENT MECHANIC","2015"],["ALBRIGHT,DENISE M","08","GS","$60,404.00","$132.00","SOCIAL SECURITY ADMINISTRATION","BALTIMORE","LEGAL ASSISTANCE","2015"],["ALBRIGHT,DOROTHY","13","GS","$100,161.00","$0.00","FOREST SERVICE","CORVALLIS","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBRIGHT,DOUGLAS C","13","GS","$112,679.00","$0.00","FOOD AND DRUG ADMINISTRATION","CINCINNATI","CHEMISTRY","2015"],["ALBRIGHT,DUSTIN S.","12","GS","$84,234.00","$0.00","BUREAU OF RECLAMATION","BISMARCK","CIVIL ENGINEERING","2015"],["ALBRIGHT,DWIGHT E","10","WG","$55,410.00","$0.00","CUSTOMS AND BORDER PROTECTION","SONOITA","MAINTENANCE MECHANIC","2015"],["ALBRIGHT,ERIN E","13","GS","$97,378.00","$1,250.00","RISK MANAGEMENT AGENCY","KANSAS CITY","GENERAL BUSINESS AND INDUSTRY","2015"],["ALBRIGHT,ESTELLE","14","VP","$128,807.00","$0.00","VETERANS HEALTH ADMINISTRATION","INDIANAPOLIS","PODIATRIST","2015"],["ALBRIGHT,GEORGE A II","13","GS","$91,814.00","$500.00","DEP ASST SEC FOR INFO AND TECHNOLOGY","LAS VEGAS","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBRIGHT,GEORGE G","15","GS","$158,700.00","$0.00","HEADQUARTERS, NASA","WASHINGTON","GENERAL ENGINEERING","2015"],["ALBRIGHT,GEORGE HAVARD","PH","AD","$200,990.00","$0.00","VETERANS HEALTH ADMINISTRATION","NEW ORLEANS","MEDICAL OFFICER","2015"],["ALBRIGHT,J ORLANDO","00","ED","$132,128.00","$0.00","ADMINISTRATION FOR CHILDREN AND FAMILIES","WASHINGTON","SOCIAL SCIENCE","2015"],["ALBRIGHT,JACK G","15","GS","$151,496.00","$0.00","OFC OF SEC OF TRANSPORTATION","WASHINGTON","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBRIGHT,JAMES RAY","GI","AT","$90,217.00","$300.00","FEDERAL AVIATION ADMINISTRATION","SOUTH BEND","AIR TRAFFIC CONTROL","2015"],["ALBRIGHT,JASON E","15","GS","$131,492.00","$500.00","RISK MANAGEMENT AGENCY","KANSAS CITY","PROGRAM MANAGEMENT","2015"],["ALBRIGHT,JASON L","12","GS","$78,924.00","$0.00","NATIONAL PARK SERVICE","HARPERS FERRY","CONTRACTING","2015"],["ALBRIGHT,JEFFREY B","09","GS","$52,655.00","$1,250.00","NAT OCEANIC AND ATMOSPHERIC ADMIN","SAN DIEGO","METEOROLOGY","2015"],["ALBRIGHT,JEFFREY S.","13","GS","$116,453.00","$0.00","NATIONAL PARK SERVICE","FORT COLLINS","HYDROLOGY","2015"],["ALBRIGHT,JENNIFER L","05","GS","$37,233.00","$100.00","VETERANS HEALTH ADMINISTRATION","SAINT CLOUD","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBRIGHT,JOEL DOUGLAS","14","GS","$116,442.00","$1,310.00","BUREAU OF LAND MANAGEMENT","LAKEWOOD","PROGRAM MANAGEMENT","2015"],["ALBRIGHT,JOHN D.","15","GS","$152,611.00","$0.00","LYNDON B. JOHNSON SPACE CENTER","HOUSTON","AEROSPACE ENGINEERING","2015"],["ALBRIGHT,KAREN C","00","AD","$55,273.00","$0.00","VETERANS HEALTH ADMINISTRATION","BIRMINGHAM","MEDICAL OFFICER","2015"],["ALBRIGHT,KAREN J","09","GS","$62,931.00","$0.00","VETERANS HEALTH ADMINISTRATION","PORTLAND","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBRIGHT,KEITH A","13","GS","$108,987.00","$1,500.00","BUREAU OF THE CENSUS","SUITLAND","MATHEMATICAL STATISTICS","2015"],["ALBRIGHT,KENNETH E","13","GS","$89,032.00","$183.00","HEADQUARTERS, NASA","BAY SAINT LOUIS","CONTRACTING","2015"],["ALBRIGHT,KEVIN W.","12","GS","$74,873.00","$0.00","NATIONAL PARK SERVICE","MOOSE","PARK RANGER","2015"],["ALBRIGHT,KIMBERLY L","01","VN","$80,000.00","$0.00","VETERANS HEALTH ADMINISTRATION","LAS VEGAS","NURSE","2015"],["ALBRIGHT,KIMBERLY R","FH","AT","$92,047.00","$0.00","FEDERAL AVIATION ADMINISTRATION","ASHEVILLE","AIR TRAFFIC CONTROL","2015"],["ALBRIGHT,KIMBERLY R","02","VN","$72,861.00","$0.00","VETERANS HEALTH ADMINISTRATION","FARGO","NURSE","2015"],["ALBRIGHT,KIMBERLY T","11","GS","$100,888.00","$0.00","VETERANS HEALTH ADMINISTRATION","PALO ALTO","PHYSICAL THERAPIST","2015"],["ALBRIGHT,KRISTEN M","07","GS","$39,570.00","$0.00","OFC SEC HEALTH AND HUMAN SERVICES","SCRANTON","PHARMACY TECHNICIAN","2015"],["ALBRIGHT,LAUREN","13","GS","$86,250.00","$0.00","OFFICE OF PERSONNEL MANAGEMENT","KANSAS CITY","HUMAN RESOURCES MANAGEMENT","2015"],["ALBRIGHT,LEE B.","13","GS","$100,161.00","$0.00","U.S. FISH AND WILDLIFE SERVICE","MOFFIT","WILDLIFE REFUGE MANAGEMENT","2015"],["ALBRIGHT,LISA C","14","GS","$115,072.00","$0.00","VETERANS HEALTH ADMINISTRATION","SALT LAKE CITY","GENERAL HEALTH SCIENCE","2015"],["ALBRIGHT,LISA M E","H","SV","$94,264.00","$0.00","TRANSPORTATION SECURITY ADMINISTRATION","ARLINGTON","FINANCIAL ADMINISTRATION AND PROGRAM","2015"],["ALBRIGHT,LOIS J","02","VN","$92,838.00","$0.00","VETERANS HEALTH ADMINISTRATION","ASHEVILLE","NURSE","2015"],["ALBRIGHT,M ASHLEY","08","GS","$49,529.00","$0.00","FOOD SAFETY AND INSPECTION SERVICE","SOUTH SAINT PAUL","CONSUMER SAFETY INSPECTION","2015"],["ALBRIGHT,MARGARET I","15","GS","$138,622.00","$1,087.00","DEPARTMENT OF ENERGY","VANCOUVER","ELECTRICAL ENGINEERING","2015"],["ALBRIGHT,MARK C","11","GS","$80,716.00","$349.00","AGRICULTURAL MARKETING SERVICE","WASHINGTON","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBRIGHT,MARTIN R","12","GS","$85,379.00","$0.00","OFFICE OF WORKERS' COMPENSATION PROGRAMS","LAKEWOOD","WORKER'S COMPENSATION CLAIMS EXAMINING","2015"],["ALBRIGHT,MEGAN L","14","GS","$107,325.00","$0.00","DEP ASST SEC FOR PROGRAM AND DATA ANALYS","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBRIGHT,NATALIE R","12","GS","$78,924.00","$0.00","CENTERS FOR MEDICARE AND MEDICAID SERVIC","BETHESDA","HEALTH INSURANCE ADMINISTRATION","2015"],["ALBRIGHT,PATRICIA A","11","GS","$84,800.00","$200.00","SOCIAL SECURITY ADMINISTRATION","TORRANCE","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALBRIGHT,RANDY L","05","GS","$35,140.00","$0.00","VETERANS HEALTH ADMINISTRATION","AMARILLO","MEDICAL SUPPORT ASSISTANCE","2015"],["ALBRIGHT,RANDY S","04","GS","$28,553.00","$0.00","FOREST SERVICE","NEMO","FORESTRY TECHNICIAN","2015"],["ALBRIGHT,RAYMOND C.","14","GS","$121,648.00","$0.00","NATIONAL PARK SERVICE","KNOXVILLE","GEN NATURAL RESOURCES MGT AND BIO SCI","2015"],["ALBRIGHT,RAYMOND M","14","GS","$125,213.00","$0.00","IMMIGRATION AND CUSTOMS ENFORCEMENT","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBRIGHT,RICHARD G.","00","ES","$165,883.00","$0.00","ENVIRONMENTAL PROTECTION AGENCY","SEATTLE","PROGRAM MANAGEMENT","2015"],["ALBRIGHT,RICHARD WYNN","11","WG","$64,300.00","$0.00","NATIONAL INSTITUTES OF HEALTH","HAMILTON","UTILITY SYSTEMS REPAIRING-OPERATING","2015"],["ALBRIGHT,ROBERT M","01","AD","$53,969.00","$150.00","VETERANS HEALTH ADMINISTRATION","NASHVILLE","NURSE","2015"],["ALBRIGHT,RYAN T","09","GS","$54,423.00","$0.00","VETERANS HEALTH ADMINISTRATION","BALTIMORE","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBRIGHT,SABRINA D","07","GS","$51,666.00","$306.00","NATURAL RESOURCES CONSERVATION SERVICE","HAGERSTOWN","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBRIGHT,SAMANTHA M","09","GS","$52,668.00","$0.00","FOOD AND DRUG ADMINISTRATION","SILVER SPRING","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBRIGHT,SAMUEL SCOTT","05","GS","$31,944.00","$0.00","GEOLOGICAL SURVEY","SAN JUAN","HYDROLOGIC TECHNICIAN","2015"],["ALBRIGHT,SARA L","07","GS","$51,437.00","$0.00","CUSTOMS AND BORDER PROTECTION","SAULT STE MARIE","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBRIGHT,SARAH E","07","GS","$43,258.00","$0.00","VETERANS HEALTH ADMINISTRATION","PROVIDENCE","GENERAL HEALTH SCIENCE","2015"],["ALBRIGHT,SCOTT H.","14","GS","$118,057.00","$2,860.00","ENVIRONMENTAL PROTECTION AGENCY","WASHINGTON","GENERAL ATTORNEY","2015"],["ALBRIGHT,STEVEN W","14","GS","$124,003.00","$0.00","U.S. TRUSTEE PROGRAM","PITTSBURGH","AUDITING","2015"],["ALBRIGHT,SUZANNE V","11","GS","$82,840.00","$0.00","NATIONAL INSTITUTES OF HEALTH","BETHESDA","MEDICAL TECHNOLOGIST","2015"],["ALBRIGHT,TERRY LEE","05","GS","$31,944.00","$0.00","FOOD SAFETY AND INSPECTION SERVICE","CROMWELL","FOOD INSPECTION","2015"],["ALBRIGHT,TESSA S","13","GS","$108,987.00","$420.00","SOCIAL SECURITY ADMINISTRATION","WOODLAWN","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALBRIGHT,THERESA L","06","GS","$40,358.00","$250.00","VETERANS HEALTH ADMINISTRATION","TUCSON","PHARMACY TECHNICIAN","2015"],["ALBRIGHT,THOMAS A","09","GS","$58,081.00","$0.00","FOREST SERVICE","WILLIAMSPORT","FORESTRY","2015"],["ALBRIGHT,THOMAS H","13","GS","$118,069.00","$0.00","U.S. MARSHALS SERVICE","ARLINGTON","BUDGET ANALYSIS","2015"],["ALBRIGHT,THOMAS J","02","WG","$37,712.00","$0.00","VETERANS HEALTH ADMINISTRATION","BALTIMORE","CUSTODIAL WORKING","2015"],["ALBRIGHT,TIFFANY R.","12","GS","$74,873.00","$0.00","SOCIAL SECURITY ADMINISTRATION","MUSKEGON","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALBRIGHT,TONI L","06","GS","$46,792.00","$0.00","VETERANS HEALTH ADMINISTRATION","BUTLER","PRACTICAL NURSE","2015"],["ALBRIGHT,TRACEY A","12","GS","$93,736.00","$1,137.00","DEPARTMENT OF ENERGY","VANCOUVER","CONTRACTING","2015"],["ALBRIGHT,TRACI J","14","GS","$111,784.00","$280.00","DEPARTMENT OF ENERGY","BILLINGS","FINANCIAL MANAGEMENT","2015"],["ALBRIGHT,TYLER S SR","08","GS","$56,974.00","$0.00","AGRICULTURAL RESEARCH SERVICE","PEORIA","COMPUTER CLERK AND ASSISTANT","2015"],["ALBRIGHT,WAYNE F","13","GS","$107,864.00","$1,050.00","NAT OCEANIC AND ATMOSPHERIC ADMIN","WAKEFIELD","METEOROLOGY","2015"],["ALBRIGHT-KNOX,CYNTHIA","03","VN","$93,723.00","$0.00","VETERANS HEALTH ADMINISTRATION","CINCINNATI","NURSE","2015"],["ALBRIGHTSON,KRISTOFER J","02","WG","$27,861.00","$0.00","VETERANS HEALTH ADMINISTRATION","CHARLESTON","CUSTODIAL WORKING","2015"],["ALBRINCK,JENNIFER K.","13","GS","$99,171.00","$0.00","NATIONAL PARK SERVICE","TWENTYNINE PALMS","PARK RANGER","2015"],["ALBRINCK,JOSEPH M.","05","WG","$38,881.00","$0.00","NATIONAL PARK SERVICE","INTERIOR","MAINTENANCE MECHANIC","2015"],["ALBRITE,ARLENE J","06","GS","$37,983.00","$0.00","VETERANS HEALTH ADMINISTRATION","LEBANON","FINANCIAL CLERICAL AND ASSISTANCE","2015"],["ALBRITTAIN,KRISTEN W","12","GS","$81,471.00","$0.00","NAT ARCHIVES AND RECORDS ADMINISTRATION","COLLEGE PARK","ARCHIVES TECHNICIAN","2015"],["ALBRITTEN,ALGEN JR","14","GS","$128,790.00","$0.00","DHS HEADQUARTERS","WASHINGTON","SECURITY ADMINISTRATION","2015"],["ALBRITTON TURNER,CAMERO","02","GS","$25,364.00","$0.00","BUREAU OF THE CENSUS","SUITLAND","INFORMATION TECHNOLOGY STUDENT TRAINEE","2015"],["ALBRITTON,ADRIENNE E","14","GS","$125,213.00","$0.00","FED RETIREMENT THRIFT INVESTMENT BOARD","WASHINGTON","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBRITTON,AURELIA S","08","GS","$57,224.00","$0.00","U.S. AGENCY FOR INTERNATIONAL DEV","WASHINGTON","HUMAN RESOURCES ASSISTANCE","2015"],["ALBRITTON,AZALEA OATES","13","PG","$108,987.00","$0.00","GOVERNMENT PRINTING OFFICE","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBRITTON,BELINDA M","03","WG","$28,696.00","$0.00","VETERANS HEALTH ADMINISTRATION","AMARILLO","CUSTODIAL WORKING","2015"],["ALBRITTON,BRANDY J","06","GS","$35,609.00","$0.00","VETERANS BENEFITS ADMINISTRATION","LOUISVILLE","CLAIMS ASSISTANCE AND EXAMINING","2015"],["ALBRITTON,CARL A","14","GS","$125,213.00","$0.00","IMMIGRATION AND CUSTOMS ENFORCEMENT","WASHINGTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBRITTON,COURTNEY D","08","GS","$49,668.00","$0.00","VETERANS HEALTH ADMINISTRATION","DUBLIN","CONTACT REPRESENTATIVE","2015"],["ALBRITTON,DAVID Q","08","GS","$61,994.00","$200.00","VETERANS HEALTH ADMINISTRATION","MARTINSBURG","GENERAL HEALTH SCIENCE","2015"],["ALBRITTON,DAWN J","06","GS","$36,796.00","$0.00","VETERANS HEALTH ADMINISTRATION","MURFREESBORO","FINANCIAL CLERICAL AND ASSISTANCE","2015"],["ALBRITTON,DERRINGER S","05","GS","$38,335.00","$0.00","VETERANS HEALTH ADMINISTRATION","GAINESVILLE","MAIL AND FILE","2015"],["ALBRITTON,ERNEST F JR","01","VN","$65,049.00","$2,000.00","VETERANS HEALTH ADMINISTRATION","GREENVILLE","NURSE","2015"],["ALBRITTON,GWENDOLYN M","05","GS","$35,140.00","$0.00","VETERANS HEALTH ADMINISTRATION","TAMPA","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBRITTON,HAROLD LEE","05","GS","$33,009.00","$0.00","VETERANS HEALTH ADMINISTRATION","DUBLIN","VOCATIONAL REHABILITATION","2015"],["ALBRITTON,JOSHUA W.","07","GS","$40,889.00","$0.00","NATIONAL PARK SERVICE","GATLINBURG","BIOLOGICAL SCIENCE TECHNICIAN","2015"],["ALBRITTON,KIMBERLY","12","IC","$73,347.00","$0.00","FEDERAL EMERGENCY MANAGEMENT AGENCY","ATLANTA","LOGISTICS MANAGEMENT","2015"],["ALBRITTON,LOUWILDA","12","GS","$84,017.00","$0.00","PATENT AND TRADEMARK OFFICE","ALEXANDRIA","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALBRITTON,MELINDA C","12","GS","$127,086.00","$0.00","VETERANS HEALTH ADMINISTRATION","MARTINSBURG","PHARMACIST","2015"],["ALBRITTON,MELISSA D","07","GS","$47,481.00","$500.00","ANIMAL AND PLANT HEALTH INSPECTION SERV","AVON PARK","PLANT PROTECTION TECHNICIAN","2015"],["ALBRITTON,MICHAEL E","13","GS","$99,905.00","$0.00","PUBLIC BUILDINGS SERVICE","WASHINGTON","ENGINEERING TECHNICAL","2015"],["ALBRITTON,MICHAEL L","11","GS","$64,418.00","$0.00","BUREAU OF LAND MANAGEMENT","MISSOULA","GEN NATURAL RESOURCES MGT AND BIO SCI","2015"],["ALBRITTON,NINA ALICIA","12","GS","$86,564.00","$0.00","GODDARD SPACE FLIGHT CENTER","GREENBELT","CONTRACTING","2015"],["ALBRITTON,PERRY","02","WG","$39,799.00","$0.00","VETERANS HEALTH ADMINISTRATION","BRONX","FOOD SERVICE WORKING","2015"],["ALBRITTON,PETER C","12","GS","$96,750.00","$0.00","SMITHSONIAN INST (EXC SEP BRDS OF TRUST)","WASHINGTON","EXHIBITS SPECIALIST","2015"],["ALBRITTON,RYAN FRANCIS","08","WS","$62,213.00","$0.00","BUREAU OF PRISONS\/FEDERAL PRISON SYSTEM","COLEMAN","MAINTENANCE MECHANIC","2015"],["ALBRITTON,TIMOTHY L","13","GS","$98,847.00","$0.00","FEDERAL EMERGENCY MANAGEMENT AGENCY","ATLANTA","EMERGENCY MANAGEMENT SPECIALIST","2015"],["ALBRITTON,TIMOTHY PAUL","12","GS","$86,574.00","$0.00","NATURAL RESOURCES CONSERVATION SERVICE","AUBURN","FORESTRY","2015"],["ALBRIZIO,ROSE ANNE","13","GS","$122,336.00","$0.00","LYNDON B. JOHNSON SPACE CENTER","HOUSTON","ELECTRONICS ENGINEERING","2015"],["ALBRO,ELIZABETH R.","15","GS","$151,496.00","$0.00","INSTITUTE OF EDUCATION SCIENCES","WASHINGTON","EDUCATION RESEARCH","2015"],["ALBRO,JANE","04","GS","$34,617.00","$0.00","BUREAU OF THE CENSUS","ROCKY HILL","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBRO,LINDA M","13","GS","$108,987.00","$0.00","ASST SEC FOR HOUSING--FED HOUSING CMSNR","WASHINGTON","GENERAL BUSINESS AND INDUSTRY","2015"],["ALBRO,LORNE A","12","GS","$82,868.00","$0.00","VETERANS HEALTH ADMINISTRATION","GLENDALE","ADMINISTRATIVE OFFICER","2015"],["ALBRO,THOMAS P","11","GS","$66,009.00","$200.00","VETERANS BENEFITS ADMINISTRATION","BUFFALO","VETERANS CLAIMS EXAMINING","2015"],["ALBRO,TRACY E.","08","GS","$51,129.00","$0.00","U.S. FISH AND WILDLIFE SERVICE","TULELAKE","BIOLOGICAL SCIENCE TECHNICIAN","2015"],["ALBROUGH,JOHN W","12","GS","$88,613.00","$0.00","U.S. COAST GUARD","LOS ANGELES","SECURITY ADMINISTRATION","2015"],["ALBROW,EMMA GILLIAN","00","AD","$27,641.00","$0.00","VETERANS HEALTH ADMINISTRATION","LOS ANGELES","SOCIAL WORK","2015"],["ALBRYCHT,RUSSELL J","01","VN","$72,840.00","$0.00","VETERANS HEALTH ADMINISTRATION","WEST HAVEN","NURSE","2015"],["ALBU GARDNER,NIKKI E","11","GS","$74,810.00","$0.00","BUREAU OF PRISONS\/FEDERAL PRISON SYSTEM","MARIANNA","PHYSICIAN ASSISTANT","2015"],["ALBU,GEORGE M","14","GS","$124,845.00","$0.00","FED MEDIATION AND CONCILIATION SERVICE","COLUMBUS","MEDIATION","2015"],["ALBUCHER,MATTHEW S","12","GS","$79,484.00","$0.00","OFFICE OF THE HUMAN RESOURCES MANAGEMENT","CHICAGO","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALBUQUERQUE,ALLWYN J","13","GS","$112,014.00","$200.00","SOCIAL SECURITY ADMINISTRATION","WOODLAWN","INFORMATION TECHNOLOGY MANAGEMENT","2015"],["ALBUQUERQUE,F C JR","PH","AD","$287,035.00","$0.00","VETERANS HEALTH ADMINISTRATION","RICHMOND","MEDICAL OFFICER","2015"],["ALBUQUERQUE,JOSE","15","GS","$158,700.00","$4,000.00","FEDERAL COMMUNICATIONS COMMISSION","WASHINGTON","ELECTRONICS ENGINEERING","2015"],["ALBUQUERQUE,MELISSA S","13","GS","$107,569.00","$0.00","CENTERS FOR DISEASE CONTROL AND PREVENTN","ATLANTA","PUBLIC HEALTH PROGRAM SPECIALIST","2015"],["ALBUQUERQUE,PAUL","J","FV","$101,546.00","$2,200.00","FEDERAL AVIATION ADMINISTRATION","WASHINGTON","GENERAL ENGINEERING","2015"],["ALBURGER,JASON","J","FV","$131,074.00","$750.00","FEDERAL AVIATION ADMINISTRATION","ATLANTIC CITY","ELECTRONICS ENGINEERING","2015"],["ALBURO,AMANDA A","02","VN","$114,620.00","$0.00","VETERANS HEALTH ADMINISTRATION","NEW YORK","NURSE","2015"],["ALBURY,FRANCILE","05","GS","$43,942.00","$0.00","VETERANS HEALTH ADMINISTRATION","MIAMI","MEDICAL SUPPORT ASSISTANCE","2015"],["ALBURY,JUDITH R","13","GS","$103,524.00","$550.00","ADMINISTRATION FOR CHILDREN AND FAMILIES","NEW YORK","SOCIAL SCIENCE","2015"],["ALBURY,TODD-JIYAMA B","05","GS","$34,284.00","$0.00","VETERANS HEALTH ADMINISTRATION","DENVER","VOUCHER EXAMINING","2015"],["ALBUS,DANIEL J.","09","GL","$68,465.00","$0.00","NATIONAL PARK SERVICE","HANCOCK","PARK RANGER","2015"],["ALBUS,JOHN J","DE","VM","$168,634.00","$0.00","VETERANS HEALTH ADMINISTRATION","QUEENS","DENTAL OFFICER","2015"],["ALBUS,NICOLE H","13","GS","$91,814.00","$0.00","ASSISTANT SEC FOR HUMAN RESOURCES MGMT","ALTOONA","HUMAN RESOURCES MANAGEMENT","2015"],["ALBY-ARMENTA,MARGARET A","06","GS","$48,804.00","$0.00","INDIAN AFFAIRS","PORTLAND","MISCELLANEOUS CLERK AND ASSISTANT","2015"],["ALBYN,KEITH C","14","GS","$126,971.00","$0.00","GEORGE C. MARSHALL SPACE FLIGHT CENTER","REDSTONE ARSENAL","MATERIALS ENGINEERING","2015"],["ALBYN,MARY F","12","GS","$118,886.00","$0.00","VETERANS HEALTH ADMINISTRATION","DENVER","PHARMACIST","2015"],["ALCAIDA,LUCILLE T.","07","GS","$46,163.00","$0.00","INDIAN AFFAIRS","PARKER","FINANCIAL ADMINISTRATION AND PROGRAM","2015"],["ALCAIDA,WILLIAM C.","05","GS","$33,009.00","$0.00","INDIAN AFFAIRS","POSTON","IRRIGATION SYSTEM OPERATION","2015"],["ALCAIDE,JOUY C","06","GS","$54,072.00","$0.00","VETERANS HEALTH ADMINISTRATION","TACOMA","PRACTICAL NURSE","2015"],["ALCAIN,JOSEFINA S","04","GS","$37,886.00","$0.00","ANIMAL AND PLANT HEALTH INSPECTION SERV","HONOLULU","PLANT PROTECTION TECHNICIAN","2015"],["ALCAIRE,CAROL DIANA","06","GS","$45,049.00","$0.00","VETERANS HEALTH ADMINISTRATION","PORTLAND","PHARMACY TECHNICIAN","2015"],["ALCALA SALES,JAMES M II","12","GS","$79,554.00","$275.00","SOCIAL SECURITY ADMINISTRATION","DULUTH","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALCALA,ALEJANDRO","05","GS","$33,009.00","$0.00","VETERANS HEALTH ADMINISTRATION","ALBUQUERQUE","HEALTH AID AND TECHNICIAN","2015"],["ALCALA,ALEJANDRO D","09","GS","$58,189.00","$1,000.00","VETERANS HEALTH ADMINISTRATION","DURHAM","PSYCHOLOGY AID AND TECHNICIAN","2015"],["ALCALA,ANNALIZA T","06","GS","$47,778.00","$0.00","VETERANS HEALTH ADMINISTRATION","MONTEREY","MEDICAL SUPPORT ASSISTANCE","2015"],["ALCALA,BECKY A","06","GS","$39,021.00","$0.00","VETERANS HEALTH ADMINISTRATION","HINES","MEDICAL SUPPORT ASSISTANCE","2015"],["ALCALA,CHRISTINE K","02","AD","$76,749.00","$0.00","VETERANS HEALTH ADMINISTRATION","ALBUQUERQUE","NURSE","2015"],["ALCALA,ELISA C","12","GS","$81,365.00","$246.00","SOCIAL SECURITY ADMINISTRATION","GLENDALE","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALCALA,EMPERATRIZ","06","GS","$52,199.00","$0.00","VETERANS HEALTH ADMINISTRATION","KINGS","HEALTH AID AND TECHNICIAN","2015"],["ALCALA,FELIX T","06","WG","$43,264.00","$0.00","NATIONAL CEMETERY ADMINISTRATION","MARION","CEMETERY CARETAKING","2015"],["ALCALA,GRACE B","06","GS","$45,502.00","$0.00","VETERANS HEALTH ADMINISTRATION","HOUSTON","DENTAL ASSISTANT","2015"],["ALCALA,JENNIFER N","PH","VM","$274,487.00","$0.00","VETERANS HEALTH ADMINISTRATION","TUCSON","MEDICAL OFFICER","2015"],["ALCALA,JOSE","09","GS","$56,468.00","$0.00","VETERANS HEALTH ADMINISTRATION","ORLANDO","MEDICAL TECHNOLOGIST","2015"],["ALCALA,JOSE C JR","06","WG","$36,397.00","$0.00","DEP ASST SEC FOR INFO AND TECHNOLOGY","AUSTIN","MATERIALS HANDLER","2015"],["ALCALA,JUAN J JR","04","GS","$41,349.00","$0.00","SOCIAL SECURITY ADMINISTRATION","OXNARD","CONTACT REPRESENTATIVE","2015"],["ALCALA,LISA C","12","GS","$74,873.00","$0.00","VETERANS HEALTH ADMINISTRATION","GAINESVILLE","SOCIAL WORK","2015"],["ALCALA,MARCOS J.","08","WG","$56,495.00","$0.00","U.S. FISH AND WILDLIFE SERVICE","AUSTWELL","MAINTENANCE MECHANIC","2015"],["ALCALA,MARGARITA","07","GS","$44,844.00","$0.00","FOOD SAFETY AND INSPECTION SERVICE","DODGE CITY","FOOD INSPECTION","2015"],["ALCALA,MICHAEL J","12","GS","$72,533.00","$0.00","FOOD AND DRUG ADMINISTRATION","NOGALES","CONSUMER SAFETY","2015"],["ALCALA,MYLENE S","02","VN","$109,450.00","$0.00","VETERANS HEALTH ADMINISTRATION","NEW YORK","NURSE","2015"],["ALCALA,PENELOPE","FH","AT","$83,257.00","$0.00","FEDERAL AVIATION ADMINISTRATION","ISLIP","AIR TRAFFIC CONTROL","2015"],["ALCALA,RAYMOND ANTHONY","05","GS","$31,944.00","$0.00","VETERANS HEALTH ADMINISTRATION","TOPEKA","CONTACT REPRESENTATIVE","2015"],["ALCALA,REGINA D","J","FV","$133,106.00","$900.00","FEDERAL AVIATION ADMINISTRATION","FORT WORTH","HUMAN RESOURCES MANAGEMENT","2015"],["ALCALA,SANDRA M","03","VN","$107,787.00","$0.00","VETERANS HEALTH ADMINISTRATION","HINES","NURSE","2015"],["ALCALA,SARAH B","11","GS","$74,835.00","$600.00","SOCIAL SECURITY ADMINISTRATION","YONKERS","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALCALA,STEVEN G","05","GS","$35,140.00","$0.00","VETERANS HEALTH ADMINISTRATION","BIG SPRING","SOCIAL SCIENCE AID AND TECHNICIAN","2015"],["ALCALAY,NEAL","11","GS","$66,026.00","$0.00","FOOD AND DRUG ADMINISTRATION","HOUSTON","CHEMISTRY","2015"],["ALCALDE,JENNIFER LYNN","08","GS","$43,823.00","$0.00","FOOD SAFETY AND INSPECTION SERVICE","CHARLESTON","CONSUMER SAFETY INSPECTION","2015"],["ALCALDE,JOHN","11","GS","$61,963.00","$410.00","SOCIAL SECURITY ADMINISTRATION","MIAMI","SOCIAL INSURANCE ADMINISTRATION","2015"],["ALCALDE,LUIS A","07","GS","$42,207.00","$0.00","SMALL BUSINESS ADMINISTRATION","EL PASO","GENERAL BUSINESS AND INDUSTRY","2015"],["ALCALDE,NANCY T","15","GS","$158,700.00","$0.00","ST. LAWRENCE SEAWAY DEV CORP","WASHINGTON","MISCELLANEOUS ADMINISTRATION AND PROGRAM","2015"],["ALCALDE,ROBERT I.","10","WG","$62,965.00","$500.00","NATIONAL PARK SERVICE","VIENNA","ELECTRICIAN","2015"],["ALCALDE,SHARON M.","09","GS","$61,444.00","$0.00","GEOLOGICAL SURVEY","RESTON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALCAMO,THOMAS S.","14","GS","$140,511.00","$2,500.00","ENVIRONMENTAL PROTECTION AGENCY","CHICAGO","CHEMICAL ENGINEERING","2015"],["ALCANIZ,PATRICIA A","13","GS","$105,725.00","$0.00","OFFICE OF THE INSPECTOR GENERAL","TUCSON","MANAGEMENT AND PROGRAM ANALYSIS","2015"],["ALCANTAR TORRES,CESAR","06","GS","$39,171.00","$0.00","FOREST SERVICE","SPRINGVILLE","FORESTRY TECHNICIAN","2015"],["ALCANTAR TORRES,JOSE J","05","GS","$31,944.00","$0.00","FOREST SERVICE","PORTERVILLE","FORESTRY TECHNICIAN","2015"],["ALCANTAR,ADRIAN","05","GS","$35,334.00","$0.00","FOREST SERVICE","SOUTH LAKE TAHOE","FORESTRY]]
def lens(listoflists):
    count = 0
    for i in listoflists:
        count += 1
        
    return count
    

print lens(l)

SyntaxError: EOL while scanning string literal (<ipython-input-2-66c4de7f9fb3>, line 1)

In [6]:
from jinja2 import Template
import psycopg2
import os
import time
import json
import requests
import logging

num_display = 500
TABLE_NAME = "federal_salaries"
next_iter = 100000

base_url = Template(
                    'http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&s'
                    'Echo=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart={{next_record_start}}&i'
                    'DisplayLength={{num_display}}&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1'
                    '=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&'
                    'mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&'
                    'bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true'
                    '&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082'
                    )

def get_paged_table_data(next_iter):
    url = base_url.render(next_record_start=next_iter, num_display=num_display)
    generated_url = requests.get(url)
    get_data = json.loads(generated_url.text)
    data = get_data['aaData']
    return data

raw_data = get_paged_table_data(next_iter)

print raw_data

[[u'BREWER,ALFREDA', u'03', u'FE', u'$166,329.00', u'$0.00', u'U.S. AGENCY FOR INTERNATIONAL DEV', u'', u'PROGRAM MANAGEMENT', u'2015'], [u'BREWER,ALLEN D', u'11', u'GS', u'$76,948.00', u'$0.00', u'BUREAU OF LAND MANAGEMENT', u'BUFFALO', u'ENGINEERING TECHNICAL', u'2015'], [u'BREWER,ALLISON C', u'11', u'GS', u'$67,971.00', u'$0.00', u'VETERANS HEALTH ADMINISTRATION', u'WASHINGTON', u'DIETITIAN AND NUTRITIONIST', u'2015'], [u'BREWER,AMELIA J', u'03', u'VN', u'$96,624.00', u'$0.00', u'VETERANS HEALTH ADMINISTRATION', u'RUTHERFORDTON', u'NURSE', u'2015'], [u'BREWER,ANDREA D', u'03', u'VN', u'$99,683.00', u'$0.00', u'VETERANS HEALTH ADMINISTRATION', u'MOUNT VERNON', u'NURSE', u'2015'], [u'BREWER,ANGELA D', u'06', u'WG', u'$44,474.00', u'$0.00', u'VETERANS HEALTH ADMINISTRATION', u'HINES', u'COOKING', u'2015'], [u'BREWER,ANGELA S', u'04', u'GS', u'$37,121.00', u'$0.00', u'BUREAU OF THE CENSUS', u'JEFFERSONVILLE', u'STATISTICAL ASSISTANT', u'2015'], [u'BREWER,ANGELICA T.', u'08', u'GS', u'$5